## Do full evaluation on Uchoa benchmark

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from copy import deepcopy
import os
import json
import numpy as np
import pandas as pd

from lib.problems import ProblemDataset
from baselines.utils import eval_method
from baselines.VRP import methods_registry
from baselines.VRP.methods_registry import CUDA_METHODS
from lib.ltr.utils import load_model
from lib.ltr.vrp.method import RouteCKMeans

In [ ]:
SEED = 1
NSEEDS = 1 #3
CUDA = False
K_NOT_KNOWN = False
STRICT_K = False
MIN_K = True    # minimize K in case it cannot be directly provided
CORES = 4
PART = 3

# A construction heuristic should be fast!
if PART == 1:
    T_LIM = 300
elif PART == 2:
    T_LIM = 600
else:
    T_LIM = 900

SAVE_DIR = f"./outputs_eval/uchoa/"
BL_DIR = os.path.join(SAVE_DIR, "baselines")
M_DIR = os.path.join(SAVE_DIR, "model")
INF = float("inf")

DS_PTH = f"data/VRP/uchoa/n{PART}.npz"
CKPT = "outputs/final/vrp_200/gnn_pool_pointwise/2023-01-17_11-45-47_011948/checkpoints/epoch=117_val_acc=0.9592.ckpt"

In [ ]:
metrics = {}
RESULTS = {}
seeds = [SEED+i for i in range(NSEEDS)]
ds = ProblemDataset(problem="VRP", seed=SEED, data_pth=DS_PTH)
ds = ds.sample(allow_pickle=True) #, sample_size=10)

In [ ]:
ds.data


The Uchoa Benchmark reports the target number K of vehicles to be found to solve each instance.
Some construction methods can take the value of K directly into account, others at least provide the option
to minimize the number of vehicles as main objective before minimizing the total route length.
In contrast, recent SotA machine learning based methods do not take this important constraint into account,
and thus are solving a simpler, relaxed version of the problem most often finding solutions
which are not feasible for the original problem. This fact was already stated in
"Thyssens et al., Supervised Permutation Invariant Networks for Solving the CVRP with Bounded Fleet Size."
As we show in our experiments, our method is able to solve most instances while taking that constraint into account.

In [ ]:
mthd = "cw_savings"
result, smry = eval_method(
    method=getattr(methods_registry, "savings"),
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=False,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    savings_function="clarke_wright",
    min_k=MIN_K,
)
m_id = mthd
RESULTS[m_id] = result
metrics[m_id] = smry
print(smry)


In [ ]:
mthd = "sweep"
result, smry = eval_method(
    method=getattr(methods_registry, mthd),
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=False,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    min_k=MIN_K,
)
m_id = mthd
RESULTS[m_id] = result
rnd_res = deepcopy(result)

costs = np.array([r['tot_center_dist'] for r in rnd_res])
max_cost = costs[costs != INF].max()
costs = costs.reshape(NSEEDS, -1)
is_inf = np.all(costs == INF, axis=0)
print(f"inf: {is_inf.sum()}")
# buffer sweep cost mean as drop in replacement
sweep_mean_cost = np.nanmean(costs, axis=0)
sweep_mean_cost[is_inf] = max_cost
smry['center_dist_mean'] = sweep_mean_cost.mean()
print(smry)
metrics[m_id] = smry


In [ ]:
mthd = "sweep_gurobi"
result, smry = eval_method(
    method=getattr(methods_registry, mthd),
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=False,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    min_k=MIN_K,
    t_lim=T_LIM
)
m_id = mthd
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)
for i, c_rnd in enumerate(sweep_mean_cost):
    inst_cost = costs[:, i]
    inf_msk = inst_cost == INF
    if np.any(inf_msk):
        inst_cost[inf_msk] = c_rnd
        costs[:, i] = inst_cost

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry


In [ ]:
mthd = "sweep_petal"
result, smry = eval_method(
    method=getattr(methods_registry, mthd),
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=False,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    #min_k=MIN_K,
    t_lim=T_LIM
)
m_id = mthd
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)
for i, c_rnd in enumerate(sweep_mean_cost):
    inst_cost = costs[:, i]
    inf_msk = inst_cost == INF
    if np.any(inf_msk):
        inst_cost[inf_msk] = c_rnd
        costs[:, i] = inst_cost

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry

In [ ]:
mthd = "gap"
result, smry = eval_method(
    method=getattr(methods_registry, mthd),
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=False,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    #min_k=MIN_K,
    t_lim=T_LIM
)
m_id = mthd
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)
for i, c_rnd in enumerate(sweep_mean_cost):
    inst_cost = costs[:, i]
    inf_msk = inst_cost == INF
    if np.any(inf_msk):
        inst_cost[inf_msk] = c_rnd
        costs[:, i] = inst_cost

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry


In [ ]:
mthd = "pomo_greedy"
result, smry = eval_method(
    method=getattr(methods_registry, 'pomo'),
    ckpt_pth="baselines/VRP/POMO/POMO/cvrp/result/Saved_CVRP100_Model/uchoa/checkpoint-8100.pt",
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=CUDA,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    single_trajectory=True,
    augment=False,
)
m_id = mthd
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)
for i, c_rnd in enumerate(sweep_mean_cost):
    inst_cost = costs[:, i]
    inf_msk = inst_cost == INF
    if np.any(inf_msk):
        inst_cost[inf_msk] = c_rnd
        costs[:, i] = inst_cost

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry


In [ ]:
mthd = "pomo_samp"
result, smry = eval_method(
    method=getattr(methods_registry, 'pomo'),
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=BL_DIR,
    cuda=CUDA,
    k_not_known=K_NOT_KNOWN,
    sample_cfg={},
    method_str=mthd,
    verbose=False,
    num_cores=CORES,
    single_trajectory=False,
    augment=True,
)
m_id = mthd
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)
for i, c_rnd in enumerate(sweep_mean_cost):
    inst_cost = costs[:, i]
    inf_msk = inst_cost == INF
    if np.any(inf_msk):
        inst_cost[inf_msk] = c_rnd
        costs[:, i] = inst_cost

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry


In [ ]:
# # greedily assigns the last 'opt_last_frac' fraction of total nodes
# # ordered by their weight to the closest center
# # and decides convergence based on inertia
mthd = "ckmeans_greedy"
model = load_model("vrp", CKPT)

ckmeans = RouteCKMeans(
    max_iter=40,
    num_init=8,
    model=model,
    seed=SEED,
    nbh_knn=25,
    init_method="ckm++",
    convergence_criterion="inertia",
    permute_k=False,
    tol=0.001,
    verbose=False,
    opt_last_frac=0.2,
    opt_last_samples=1, # no multiple samples
    depot_priority=True,
    depot_priority_add=True,
)

result, smry = eval_method(
    method=ckmeans.inference,
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=M_DIR,
    cuda=False,
    k_not_known=K_NOT_KNOWN,
    method_str=mthd,
    min_k=False,
    k_search_bs=4,
    k_search_init=1,
    #num_cores=1,
)
m_id = f"{mthd}{'_cuda' if CUDA and mthd in CUDA_METHODS else ''}"
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry


In [ ]:
# samples multiple assignments for the last 'opt_last_frac' fraction of total nodes
# and selects the best one
mthd = "ckmeans_samp"
model = load_model("vrp", CKPT)

ckmeans = RouteCKMeans(
    max_iter=50,
    num_init=8,
    model=model,
    seed=SEED,
    nbh_knn=25,
    init_method="ckm++",
    convergence_criterion="inertia",
    permute_k=False,
    tol=0.001,
    verbose=False,
    opt_last_frac=0.2,
    opt_last_samples=16,
    depot_priority=True,
    depot_priority_add=True,
)

result, smry = eval_method(
    method=ckmeans.inference,
    dataset=ds,
    problem="cvrp",
    strict_feasibility=STRICT_K,
    seeds=seeds,
    save_dir=M_DIR,
    cuda=CUDA,
    k_not_known=K_NOT_KNOWN,
    method_str=mthd,
    min_k=False,
    k_search_bs=4,
    k_search_init=1,
    #num_cores=1,
)
m_id = f"{mthd}{'_cuda' if CUDA and mthd in CUDA_METHODS else ''}"
RESULTS[m_id] = result
# replace infeasible runs with mean cost of random method
res = deepcopy(result)
costs = np.array([r['tot_center_dist'] for r in res])
costs = costs.reshape(NSEEDS, -1)

smry['center_dist_mean'] = np.mean(costs)
smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
print(f"adapted summary: {smry}")
metrics[m_id] = smry


In [ ]:
# convert to dataframe for nice table ;)
metric_df = pd.DataFrame(metrics)
metric_df


In [ ]:
save_metrics = metric_df.to_dict()
os.makedirs(SAVE_DIR, exist_ok=True)
file = os.path.join(SAVE_DIR, f"full_results.json")
sv = True
if os.path.exists(file):
    sv = False
    inp = input("File exists! Overwrite? (y/n)")
    if str(inp).lower() == "y":
        sv = True
if sv:
    with open(file, 'w') as fp:
        json.dump(save_metrics, fp)

In [ ]:
# optimal ks defined in instances
n1_ks = [
25,
14,
13,
10,
6,
30,
18,
13,
10,
7,
46,
22,
13,
11,
10,
51,
26,
23,
15,
8,
51,
36,
19,
16,
11,
73,
34,
23,
16,
14,
48,
50,
]
print(len(n1_ks), np.mean(n1_ks))

n2_ks = [
28,
16,
13,
58,
35,
28,
17,
15,
60,
50,
31,
21,
13,
71,
53,
28,
20,
15,
84,
43,
40,
29,
17,
94,
52,
38,
29,
19,
130,
61,
37,
29,
26,
138,
70,
59,
]
print(len(n2_ks), np.mean(n2_ks))

n3_ks = [
39,
21,
153,
96,
50,
42,
30,
159,
92,
62,
43,
35,
131,
130,
75,
44,
35,
159,
98,
71,
48,
40,
171,
142,
95,
59,
37,
207,
151,
87,
58,
43,
]
print(len(n3_ks), np.mean(n3_ks))